In [1]:
import re
from fractions import Fraction
import unicodedata
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
sys.path.append("..")
from src.data import data_cleaning_util
from src.features import create_features

In [68]:
# Load raw data
input_data = pd.read_csv(
    "../data/raw/nyt-ingredients-snapshot-2015.csv", index_col="index"
)
input_data.head()

,input,name,qty,range_end,unit,comment
index,,,,,,
0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN


In [69]:
input_data.name.unique()

array(['butternut squash', 'chestnuts', 'onion', ..., 'Cojita cheese',
       'Orange Bitters', 'Brandied cherry'], dtype=object)

In [70]:
def clean_nyt_html(row, verbose=False):
    """
    This will replace all html tags that were not stripped
    from the NYT data
    """
    columns = ["input", "name", "comment"]
    for col in columns:
        # This filters out NaN values so they wont get caught in the try except
        if row[col] == row[col]:
            try:
                # this will remove all: <a href=...>see recipe</a>
                match = re.findall(r"\s*\(?<.*see\s*recipe.*>\)?", row[col])
                if match:
                    for m in match:
                        row[col] = re.sub(r"\s*\(?<.*see\s*recipe.*>\)?", "", row[col])
                        if col == "input" and row["comment"] == row["comment"]:
                            row["comment"] = re.sub(r"see recipe", "", row["comment"])
            except TypeError:
                print("ERROR: Removing <see recipe>, " + col + " ", row)
            try:
                # this will remove all: see <a href=...>recipe</a>
                match = re.findall(r"\(?\s*(see)\s*?<.*recipe.*>\)?", row[col])
                if match:
                    for m in match:
                        row[col] = re.sub(r"\(?\s*(see)\s*?<.*recipe.*>\)?", "", row[col])
                        if col == "input" and row["comment"] == row["comment"]:
                            row["comment"] = re.sub(r"see recipe", "", row["comment"])
            except TypeError:
                print("ERROR: Removing see <recipe>, " + col + " ", row)

            # This will remove all <span> and misc <a href=...>...</a>
            match = re.findall(r"<.*?>", row[col])
            if match:
                for m in match:
                    row[col] = re.sub(r"<.*?>", "", row[col])
            # this will remove all un-escapped '\n' from the original html
            match = re.findall(r"\\n", row[col])
            if match:
                for m in match:
                    row[col] = re.sub(r"\\n", " ", row[col])
            # if the column is now blank becasue of what we removed, set it
            # to NaN so pandas can handle it easier
            if not row[col]:
                row[col] = float("nan")
            else:
                row[col] = row[col].strip()
    return row

In [71]:
# There are leftover html tags that need to be removed
# Has the added benefit of stripping out the extraneous "see recipe"
# from both input and comment
input_data = input_data.apply(clean_nyt_html, axis=1)

In [72]:
print(input_data.shape)
# There are over 100 lines missing an input, these are worthless and are dropped.
input_data.dropna(axis=0, subset=["input"], inplace=True)
print(input_data.shape)

(179207, 6)
(179060, 6)


In [73]:
def clean_unicode_fractions(ingredient):
    """
    Replace unicode fractions with ascii representation, preceded by a
    space.

    "1\x215e" => "1 7/8"
    """
    try:
        # match all mixed fractions with a unicode fraction (e.g. 1 ¾ or 1¾) and add them together
        match = re.findall(r"(\d+)\s?([\u2150-\u215E\u00BC-\u00BE])", ingredient)
        if match:
            for m in match:
                num = float(m[0]) + float(Fraction(unicodedata.numeric(m[1])))
                ingredient = re.sub(
                    r"(\d+)\s?([\u2150-\u215E\u00BC-\u00BE])", str(round(num, 3)), ingredient
                )

        # match all other unicode fractions
        match = re.findall(r"([\u2150-\u215E\u00BC-\u00BE])", ingredient)
        if match:
            for m in match:
                ingredient = re.sub(
                    r"([\u2150-\u215E\u00BC-\u00BE])",
                    str(round(float(Fraction(unicodedata.numeric(m))), 3)),
                    ingredient,
                )
    except TypeError:
        print("ERROR CLEANING UNICODE: ", ingredient)
    return ingredient

In [74]:
# Unicode has numerous characters to represent fractions like ¾, we remove these
input_data["input"] = input_data["input"].apply(clean_unicode_fractions)

In [75]:
def merge_fractions(s):
    """
    Merges mixed fractions: 1 2/3 => 1.67
    """
    try:
        match = re.findall(r"(\d+)\s+(\d+\/\d+)", s)
        if match:
            for m in match:
                num = float(m[0]) + float(Fraction(m[1]))
                s = re.sub(r"(\d+)\s+(\d+\/\d+)", str(round(num, 3)), s)

        match = re.findall(r"(\d+\/\d+)", s)
        if match:
            for m in match:
                num = float(Fraction(m))
                s = re.sub(r"(\d+\/\d+)", str(round(num, 3)), s)
    except ZeroDivisionError:
        print(s)
    return s

In [76]:
# Many ingredient quantities are written as 1 1/2 to represent 1.5
# The quantity label however is always written as 1.5 so we need to 
# convert these fractions so that the crf can match it
input_data["input"] = input_data["input"].apply(merge_fractions)

In [81]:
def merge_quantities(row):
    try:
        match = re.findall(r"(\d+\.?\d*)\-?[\s\-][tor]+[\s\-](\d+\.?\d*)", row["input"])
        if match:
            for m in match:
                print(row["input"])
                num = ((float(m[0])+float(m[1])) / 2)
                row["input"] = re.sub(r"(\d+\.?\d*)\-?[\s\-][tor]+[\s\-](\d+\.?\d*)", str(round(num, 3)), row["input"])
                print(row["input"])
        match = re.findall(r"(\d+)\s+(\d+\.*\d*)", row["input"])
        if match:
            for m in match:
                num = float(m[0]) * float(m[1])
                print(row["input"], row["qty"], num)
                row["input"] = re.sub(r"(\d+)\s+(\d+\.*\d*)", str(round(num, 3)), row["input"])
                if float(m[0]) == row["qty"] or float(m[1]) == row["qty"]:
                    # probably a pretty good guess that the qty was only one of these two numbers, update it with the new num
                    row["qty"] = round(num, 3)
                print(row["input"], row["qty"], num)
    except TypeError:
        print("Error Merging Ranges: ", row)
    return row
    

In [82]:
input_data.apply(merge_quantities, axis=1)

1.25 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 10.0-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 10.0-ounce package frozen squash, defrosted 1.25 10.0
1 2-pound beef tenderloin roast, cut from the large end, trimmed and tied at 0.5 -inch intervals with kitchen twine 2.0 2.0
2.0-pound beef tenderloin roast, cut from the large end, trimmed and tied at 0.5 -inch intervals with kitchen twine 2.0 2.0
1 6-inch white-corn tortilla 1.0 6.0
6.0-inch white-corn tortilla 6.0 6.0
6 kiwis (3 to 4 ounces each), peeled and cut into rounds about 0.25-inch thick
6 kiwis (3.5 ounces each), peeled and cut into rounds about 0.25-inch thick
2 28-ounce cans crushed tomatoes 56.0 56.0
56.0-ounce cans crushed tomatoes 56.0 56.0
4

3.5 to 4 cups well-flavored chicken stock
3.75 cups well-flavored chicken stock
3.5 to 4 cups well-seasoned chicken stock
3.75 cups well-seasoned chicken stock
2 to 3 tablespoons chopped fresh cilantro, optional
2.5 tablespoons chopped fresh cilantro, optional
0.75 to 1 pound rutabaga, peeled and cut into 0.75-inch cubes
0.875 pound rutabaga, peeled and cut into 0.75-inch cubes
2 center-cut salmon fillets, each weighing 8 or 9 ounces
2 center-cut salmon fillets, each weighing 8.5 ounces
1 large monkfish tail, about 2 to 2.5 pounds
1 large monkfish tail, about 2.25 pounds
3 or 4 thyme sprigs
3.5 thyme sprigs
10 to 12 blanched almonds
11.0 blanched almonds
12 ounces whole onion or 11 ounces chopped, ready-cut onion (2.25 to 2.25 cups)
12 ounces whole onion or 11 ounces chopped, ready-cut onion (2.25 cups)
3 pounds whole cauliflower or 1.5 pounds ready-cut cauliflower florets (7 to 8 cups)
3 pounds whole cauliflower or 1.5 pounds ready-cut cauliflower florets (7.5 cups)
0.125 to 0.125 tea

4 trout (about 0.75 to 1 pound each), cleaned, scaled, gills removed
4 trout (about 0.875 pound each), cleaned, scaled, gills removed
2 whole racks of lamb (1 to 1.25 pounds each)
2 whole racks of lamb (1.125 pounds each)
5 to 6 pounds seasoned croutons
5.5 pounds seasoned croutons
1 11-to-12-pound turkey
1 11.5-pound turkey
1 11.5-pound turkey 11.0 11.5
11.5-pound turkey 11.0 11.5
0.25 to 0.25 cup freshly squeezed orange juice
0.25 cup freshly squeezed orange juice
24 to 30 unsalted tortilla chips
27.0 unsalted tortilla chips
24 to 30 melba toast rounds
27.0 melba toast rounds
24 to 30 thin slices French baguette bread
27.0 thin slices French baguette bread
24 to 30 thin slices French baguette bread or unsalted round tortilla chips
27.0 thin slices French baguette bread or unsalted round tortilla chips
16 to 21 soft ladyfingers, homemade or store-bought
18.5 soft ladyfingers, homemade or store-bought
16 to 21 soft ladyfingers, store-bought or homemade
18.5 soft ladyfingers, store-boug

2 to 3 garlic cloves, slivered
2.5 garlic cloves, slivered
1 to 2 tablespoons dark rum (optional)
1.5 tablespoons dark rum (optional)
2 or 3 large garlic cloves, peeled, crushed and chopped (2 teaspoons)
2.5 large garlic cloves, peeled, crushed and chopped (2 teaspoons)
8 to 10 scallions, trimmed (with most of the green left on), cleaned and minced (1 cup)
9.0 scallions, trimmed (with most of the green left on), cleaned and minced (1 cup)
1 12-ounce bag fresh cranberries 12.0 12.0
12.0-ounce bag fresh cranberries 12.0 12.0
3 or 4 cloves garlic, peeled and thinly sliced
3.5 cloves garlic, peeled and thinly sliced
3 to 4 cups sifted all-purpose flour
3.5 cups sifted all-purpose flour
6 to 7 cups flour, as needed
6.5 cups flour, as needed
2 to 6 whole, very small white onions, peeled
4.0 whole, very small white onions, peeled
1 pound Jerusalem artichokes or sun chokes, 8 to 10
1 pound Jerusalem artichokes or sun chokes, 9.0
1 pound Jerusalem artichokes, 8 to 10
1 pound Jerusalem artichoke

3 to 4 ounces Brachetto d'Acqui wine
3.5 ounces Brachetto d'Acqui wine
0.75 cup egg whites (5 to 6)
0.75 cup egg whites (5.5)
1 to 1.25 cups bleached all-purpose flour
1.125 cups bleached all-purpose flour
1 3-to-3 1 /2 -pound piece of veal breast, with a pocket cut into it by the butcher
1 3.0 1 /2 -pound piece of veal breast, with a pocket cut into it by the butcher
1 3.0 1 /2 -pound piece of veal breast, with a pocket cut into it by the butcher 3.0 3.0
3.0 1 /2 -pound piece of veal breast, with a pocket cut into it by the butcher 3.0 3.0
5 to 6 cups veal or chicken broth
5.5 cups veal or chicken broth
4 to 6 tablespoons extra virgin olive oil, to coat bottom of pan
5.0 tablespoons extra virgin olive oil, to coat bottom of pan
2 steaks (sirloin strip, rib-eye or other), 8 to 10 ounces each and about 1 inch thick
2 steaks (sirloin strip, rib-eye or other), 9.0 ounces each and about 1 inch thick
1.5 to 2 cups milk
1.75 cups milk
1 or 2 tablespoons lemon juice, optional
1.5 tablespoons 

...

1 cup plus 1 to 2 tablespoons sugar
1 cup plus 1.5 tablespoons sugar
1 3.5- to 4-pound whole chicken
1 3.75-pound whole chicken
1 3.75-pound whole chicken 1.0 3.75
3.75-pound whole chicken 3.75 3.75
1 to 2 tablespoons cracked black pepper, to taste
1.5 tablespoons cracked black pepper, to taste
6 to 8 curry leaves
7.0 curry leaves
3 or 4 small green chiles, finely chopped
3.5 small green chiles, finely chopped
Fine sea salt, to taste (0.5 to 1 teaspoon)
Fine sea salt, to taste (0.75 teaspoon)
2 to 3 serrano or jalapeño chiles, seeded for a milder sauce, and chopped
2.5 serrano or jalapeño chiles, seeded for a milder sauce, and chopped
1 to 2 tablespoons grapeseed oil
1.5 tablespoons grapeseed oil
1 generous bunch Swiss chard (about 0.75 pound), stemmed, leaves washed in 2 changes water, and chopped (7 to 8 cups chopped greens)
1 generous bunch Swiss chard (about 0.75 pound), stemmed, leaves washed in 2 changes water, and chopped (7.5 cups chopped greens)
2 to 3 tablespoons olive oil, a

0.333 to 0.333 large grilled or roasted red pepper (see note)
0.333 large grilled or roasted red pepper (see note)
2 5- to 6-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 0.5- to 1-inch thick
2 5.5-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 5.5-inch thick
2 5.5-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 5.5-inch thick
2 5.5-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 5.5-inch thick
2 5.5-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 5.5-inch thick 2.0 11.0
11.0-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 5.5-inch thick 11.0 11.0
2 5- to 6-ounce fish fillets, like black bass, haddock, fluke, striped bass, tilefish, snapper or salmon, 0.5- to 1-inch thick
2 5.5-ounce fish fillets, li

1 whole chicken, 3.5 pounds
2 to 6 tablespoons unsalted butter, to taste
4.0 tablespoons unsalted butter, to taste
2 to 3 pounds large Yukon Gold potatoes, quartered
2.5 pounds large Yukon Gold potatoes, quartered
4 to 6 sprigs fresh basil, plus 2 tablespoons finely shredded leaves for garnish
5.0 sprigs fresh basil, plus 2 tablespoons finely shredded leaves for garnish
4 to 6 sprigs fresh basil, plus 2 tablespoons finely shredded leaves for garnish
5.0 sprigs fresh basil, plus 2 tablespoons finely shredded leaves for garnish
4. Return uncovered pan to oven and cook until chicken skin is browned, cooking liquid is reduced, and vegetables are very soft, 10 to 15 minutes more.
4. Return uncovered pan to oven and cook until chicken skin is browned, cooking liquid is reduced, and vegetables are very soft, 12.5 minutes more.
1 3- to 4-pound chicken, cut up, or 3 to 4 pounds bone-in, skin-on chicken thighs
1 3.5-pound chicken, cut up, or 3.5 pounds bone-in, skin-on chicken thighs
1 3.5-pound

1 7⁄8 cups lukewarm water 1.88 7.0
7.0⁄8 cups lukewarm water 1.88 7.0
1 15-ounce can of chickpeas 1.0 15.0
15.0-ounce can of chickpeas 15.0 15.0
1 6-ounce can of tomato paste 1.0 6.0
6.0-ounce can of tomato paste 6.0 6.0
4 or 5 large beets (about 1.75 pounds)
4.5 large beets (about 1.75 pounds)
1 large fennel bulb, or 2 or 3 smaller bulbs
1 large fennel bulb, or 2.5 smaller bulbs
8 or 9 cloves garlic
8.5 cloves garlic
1 whole bone-in turkey breast, 10 to 14 pounds
1 whole bone-in turkey breast, 12.0 pounds
4 to 8 ounces unsalted butter
6.0 ounces unsalted butter
2 15-ounce cans (3.5 cups) pumpkin purée 2.0 30.0
30.0-ounce cans (3.5 cups) pumpkin purée 30.0 30.0
1 to 2 tablespoons cold unsalted butter (optional)
1.5 tablespoons cold unsalted butter (optional)
2 1⁄2 teaspoons achiote (annatto) paste 2.0 2.0
2.0⁄2 teaspoons achiote (annatto) paste 2.0 2.0
1.75 cups pumpkin purée, homemade or canned (1 15-ounce can) 1.75 15.0
1.75 cups pumpkin purée, homemade or canned (15.0-ounce can) 1.7

,input,name,qty,range_end,unit,comment
index,,,,,,
0,1.25 cups cooked and pureed fresh butternut sq...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,1.5 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN
...,...,...,...,...,...,...
179202,0.75 oz. pineapple juice,pineapple juice,0.75,0.0,ounce,NaN
179203,1 tsp. fresh lemon juice,lemon juice,1.00,0.0,teaspoon,fresh
179204,Angostura bitters,Angostura bitters,0.00,0.0,NaN,NaN


In [63]:
input_data.qty.unique()

array([1.250e+00, 1.000e+00, 2.000e+00, 1.500e+00, 0.000e+00, 6.000e+00,
       3.300e-01, 5.000e-01, 4.000e+00, 3.000e+00, 1.200e+01, 1.300e+01,
       2.500e-01, 5.000e+00, 7.500e-01, 1.300e-01, 2.500e+00, 9.000e+00,
       1.800e+01, 5.600e+01, 8.000e+00, 3.250e+00, 6.500e+00, 1.000e+01,
       2.250e+00, 7.000e+00, 6.700e-01, 4.500e+00, 5.800e-01, 3.200e+01,
       1.700e+01, 2.800e+01, 3.000e+01, 3.500e+01, 1.670e+00, 2.000e+01,
       1.750e+00, 1.600e+01, 9.500e+00, 4.000e+01, 2.100e+01, 1.500e+01,
       2.400e+01, 5.500e+00, 6.900e+01, 1.330e+00, 1.100e+01, 8.500e+00,
       3.500e+00, 6.000e+01, 1.400e+01, 2.500e+01, 3.800e-01, 7.330e+00,
       3.300e+01, 1.130e+00, 6.800e+01, 2.900e+01, 2.600e+01, 4.500e+01,
       1.900e+01, 7.500e+00, 1.750e+01, 2.200e+01, 1.167e+01, 1.350e+01,
       6.330e+00, 3.750e+00, 1.450e+01, 2.670e+00, 2.750e+00, 1.267e+01,
       3.600e+01, 2.330e+00, 8.300e-01, 3.330e+00, 2.700e+01, 6.600e-01,
       1.150e+01, 4.250e+00, 5.700e+01, 3.400e+01, 

In [12]:
def multiply_qty(s):
    """
    Many ingredients are written in the form 2 8.5-ounce cans...
    This is both tricky for the model to parse and made worse because
    the labeled data incosistently labels the quanity as 2, 8.5, or 17.
    We want to reuce all these to a single value:
    2 8.5-ounce => 17.0-ounce
    and update the quantity label as appropriate
    """
    match = re.findall(r"(\d+)\s+(\d+\.*\d*)", s["input"])
    if match:
        for m in match:
            num = float(m[0]) * float(m[1])
            print(s["input"], s["qty"], num)
            s["input"] = re.sub(r"(\d+)\s+(\d+\.*\d*)", str(round(num, 3)), s["input"])
            if float(m[0]) == float(s["qty"]) or float(m[1]) == float(s["qty"]):
                # probably a pretty good guess that the qty was only one of these two numbers, update it with the new num
                s["qty"] = str(round(num, 3))
            print(s["input"], s["qty"], num)
    return s

In [13]:
input_data = input_data.apply(multiply_qty, axis=1)

1.25 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 10.0-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 1 10-ounce package frozen squash, defrosted 1.25 10.0
1.25 cups cooked and pureed fresh butternut squash, or 10.0-ounce package frozen squash, defrosted 1.25 10.0
1 2-pound beef tenderloin roast, cut from the large end, trimmed and tied at 0.5 -inch intervals with kitchen twine 2.0 2.0
2.0-pound beef tenderloin roast, cut from the large end, trimmed and tied at 0.5 -inch intervals with kitchen twine 2.0 2.0
1 6-inch white-corn tortilla 1.0 6.0
6.0-inch white-corn tortilla 6.0 6.0
2 28-ounce cans crushed tomatoes 56.0 56.0
56.0-ounce cans crushed tomatoes 56.0 56.0
4 9-ounce boxes frozen artichoke hearts 4.0 36.0
36.0-ounce boxes frozen artichoke hearts 36.0 36.0
2 16-ounce cans dark cherries, pitted 32.0 32.0
32.0-oun

1 28-ounce can crushed tomatoes 28.0 28.0
28.0-ounce can crushed tomatoes 28.0 28.0
2 8-ounce bags white pearl onions 16.0 16.0
16.0-ounce bags white pearl onions 16.0 16.0
2 8-ounce bags red pearl onions 16.0 16.0
16.0-ounce bags red pearl onions 16.0 16.0
2 10-ounce packages breakfast sausage 20.0 20.0
20.0-ounce packages breakfast sausage 20.0 20.0
1 4-pound chicken 1.0 4.0
4.0-pound chicken 4.0 4.0
1 4.5-ounce piece blue cheese (Danish, Stilton, Gorgonzola or Roquefort) 4.5 4.5
4.5-ounce piece blue cheese (Danish, Stilton, Gorgonzola or Roquefort) 4.5 4.5
2 12-ounce packages firm cranberries 24.0 24.0
24.0-ounce packages firm cranberries 24.0 24.0
1 14-pound turkey 1.0 14.0
14.0-pound turkey 14.0 14.0
1 15-ounce jar chestnuts packed in syrup, drained (see note) 15.0 15.0
15.0-ounce jar chestnuts packed in syrup, drained (see note) 15.0 15.0
1 3-inch strip of orange peel 1.0 3.0
3.0-inch strip of orange peel 3.0 3.0
1 4-pound duck 1.0 4.0
4.0-pound duck 4.0 4.0
1 12-ounce bag cranbe

1 13.5-ounce can unsweetened coconut milk, shaken well 13.5 13.5
13.5-ounce can unsweetened coconut milk, shaken well 13.5 13.5
1 15-ounce can no-salt-added black beans 15.0 15.0
15.0-ounce can no-salt-added black beans 15.0 15.0
1 1-inch piece fresh ginger, peeled and crushed or chopped very fine 1.0 1.0
1.0-inch piece fresh ginger, peeled and crushed or chopped very fine 1.0 1.0
4 17-ounce cans corn niblets, drained if necessary 68.0 68.0
68.0-ounce cans corn niblets, drained if necessary 68.0 68.0
1 1-inch piece ginger, unpeeled 1.0 1.0
1.0-inch piece ginger, unpeeled 1.0 1.0
1 5-inch piece fresh ginger, peeled and coarsely chopped 1.0 5.0
5.0-inch piece fresh ginger, peeled and coarsely chopped 5.0 5.0
1 3- inch piece fresh ginger, cut in small slices 1.0 3.0
3.0- inch piece fresh ginger, cut in small slices 3.0 3.0
1 3-inch piece fresh ginger, coarsely chopped 1.0 3.0
3.0-inch piece fresh ginger, coarsely chopped 3.0 3.0
1 4-inch-square piece konbu (dried kelp) 1.0 4.0
4.0-inch-sq

1 4.5 pound loin of pork, boned 4.5 4.5
4.5 pound loin of pork, boned 4.5 4.5
4 6-ounce salmon fillets 24.0 24.0
24.0-ounce salmon fillets 24.0 24.0
1 3-inch cinnamon stick 1.0 3.0
3.0-inch cinnamon stick 3.0 3.0
1 35-ounce can tomato puree 35.0 35.0
35.0-ounce can tomato puree 35.0 35.0
1 28-ounce can diced tomatoes with their juice 28.0 28.0
28.0-ounce can diced tomatoes with their juice 28.0 28.0
1 28-ounce can no-salt-added crushed tomatoes 28.0 28.0
28.0-ounce can no-salt-added crushed tomatoes 28.0 28.0
4 loin pork chops, about 2 pounds (or 4 8-ounce pork chops) 4.0 32.0
4 loin pork chops, about 2 pounds (or 32.0-ounce pork chops) 32.0 32.0
1 2.5-pound chicken, cut up, backbone and wing tips reserved for broth 2.5 2.5
2.5-pound chicken, cut up, backbone and wing tips reserved for broth 2.5 2.5
1 2.5-pound live lobster 2.5 2.5
2.5-pound live lobster 2.5 2.5
2 8-inch-long eggplants, peeled and cut into 0.5-inch-thick slices 2.0 16.0
16.0-inch-long eggplants, peeled and cut into 0.5

4 3-inch beef marrow bones 4.0 12.0
12.0-inch beef marrow bones 12.0 12.0
1 3-inch stick cinnamon 1.0 3.0
3.0-inch stick cinnamon 3.0 3.0
1 35-ounce can plum tomatoes, squeezed and drained thoroughly 35.0 35.0
35.0-ounce can plum tomatoes, squeezed and drained thoroughly 35.0 35.0
2 4-pound chickens 8.0 8.0
8.0-pound chickens 8.0 8.0
1 14.5-ounce can peeled tomatoes, well drained and coarsely chopped 14.5 14.5
14.5-ounce can peeled tomatoes, well drained and coarsely chopped 14.5 14.5
1 13.75-ounce can artichoke hearts, well drained 13.75 13.75
13.75-ounce can artichoke hearts, well drained 13.75 13.75
1 3.5-pound roasting chicken 3.5 3.5
3.5-pound roasting chicken 3.5 3.5
2 13.75-ounce cans artichoke hearts, well drained and quartered 2.0 27.5
27.5-ounce cans artichoke hearts, well drained and quartered 27.5 27.5
2 1.25-pound lobsters, steamed, tail and claw meat removed and cut into 0.5-inch pieces 2.0 2.5
2.5-pound lobsters, steamed, tail and claw meat removed and cut into 0.5-inch 

...

1 14-ounce can chopped tomatoes, with juice 1.0 14.0
14.0-ounce can chopped tomatoes, with juice 14.0 14.0
1 4-inch piece fresh ginger, peeled and sliced crosswise 0.25-inch thick 1.0 4.0
4.0-inch piece fresh ginger, peeled and sliced crosswise 0.25-inch thick 4.0 4.0
4 1.25-inch-thick) bone-in pork chops (about 0.75 pound each) 4.0 5.0
5.0-inch-thick) bone-in pork chops (about 0.75 pound each) 5.0 5.0
1 12-ounce piece sushi-grade tuna, cut into four 4-by-1.5-inch pieces 1.0 12.0
12.0-ounce piece sushi-grade tuna, cut into four 4-by-1.5-inch pieces 12.0 12.0
8 4-inch round rice crackers, ground to small crumbs 8.0 32.0
32.0-inch round rice crackers, ground to small crumbs 32.0 32.0
1 4-pound piece of pork belly, skin-on 1.0 4.0
4.0-pound piece of pork belly, skin-on 4.0 4.0
2.5 cups fresh raspberries or 1 10-ounce package frozen raspberries, defrosted 2.5 10.0
2.5 cups fresh raspberries or 10.0-ounce package frozen raspberries, defrosted 2.5 10.0
1 pound tomatoes, peeled, seeded and ch

2 3-inch pieces kombu (kelp), wiped with a damp rag 2.0 6.0
6.0-inch pieces kombu (kelp), wiped with a damp rag 6.0 6.0
1 6-ounce bag of spinach 1.0 6.0
6.0-ounce bag of spinach 6.0 6.0
1 1-to-2-inch cinnamon stick, broken into pieces, or 1 teaspoon ground cinnamon 1.0 1.0
1.0-to-2-inch cinnamon stick, broken into pieces, or 1 teaspoon ground cinnamon 1.0 1.0
3 tomatoes, peeled and roughly chopped, or 1 15-ounce can whole tomatoes 3.0 15.0
3 tomatoes, peeled and roughly chopped, or 15.0-ounce can whole tomatoes 3.0 15.0
1 2-inch piece fresh ginger 1.0 2.0
2.0-inch piece fresh ginger 2.0 2.0
1 4-inch Chinese long red hot chile, or to taste 1.0 4.0
4.0-inch Chinese long red hot chile, or to taste 4.0 4.0
1 4-inch serrano chile, or to taste 1.0 4.0
4.0-inch serrano chile, or to taste 4.0 4.0
1 generous bunch spinach (about 0.75 pound), stemmed, washed thoroughly and coarsely chopped, or 2 6-ounce bags baby spinach 1.0 12.0
1 generous bunch spinach (about 0.75 pound), stemmed, washed thoro

1 29-ounce can white hominy, drained and rinsed 1.0 29.0
29.0-ounce can white hominy, drained and rinsed 29.0 29.0
1 29-ounce can white hominy, drained and rinsed 1.0 29.0
29.0-ounce can white hominy, drained and rinsed 29.0 29.0
1 28-ounce can chopped tomatoes in juice, pulsed in a food processor 1.0 28.0
28.0-ounce can chopped tomatoes in juice, pulsed in a food processor 28.0 28.0
1 28-ounce can tomatoes in juice 1.0 28.0
28.0-ounce can tomatoes in juice 28.0 28.0
1 2-inch cinnamon stick 2.0 2.0
2.0-inch cinnamon stick 2.0 2.0
1 12-ounce bunch spinach, stemmed and washed, or 1 6-ounce bag baby spinach 1.0 12.0
12.0-ounce bunch spinach, stemmed and washed, or 12.0-ounce bag baby spinach 12.0 12.0
12.0-ounce bunch spinach, stemmed and washed, or 12.0-ounce bag baby spinach 12.0 6.0
12.0-ounce bunch spinach, stemmed and washed, or 12.0-ounce bag baby spinach 12.0 6.0
1 large bunch spinach, stemmed, or 1 6-ounce bag baby spinach 1.0 6.0
1 large bunch spinach, stemmed, or 6.0-ounce bag b

8 8-inch flour or corn tortillas, preferably homemade  ) 8.0 64.0
64.0-inch flour or corn tortillas, preferably homemade  ) 64.0 64.0
1 generous bunch spinach, stemmed and washed well in 2 changes water, or 2 6-ounce bags baby spinach, washed 1.0 12.0
1 generous bunch spinach, stemmed and washed well in 2 changes water, or 12.0-ounce bags baby spinach, washed 1.0 12.0
12 6-inch corn tortillas 12.0 72.0
72.0-inch corn tortillas 72.0 72.0
1 28-ounce can whole peeled tomatoes 1.0 28.0
28.0-ounce can whole peeled tomatoes 28.0 28.0
1 3-pound chicken 1.0 3.0
3.0-pound chicken 3.0 3.0
2 16-ounce cans yellow hominy (do not use white) 2.0 32.0
32.0-ounce cans yellow hominy (do not use white) 32.0 32.0
2 10-ounce packages whole-leaf frozen spinach (do not thaw) 2.0 20.0
20.0-ounce packages whole-leaf frozen spinach (do not thaw) 20.0 20.0
1 15-ounce can chick peas, drained and rinsed 1.0 15.0
15.0-ounce can chick peas, drained and rinsed 15.0 15.0
1 14-ounce block of organic extra-firm tofu (fi

1 3-to-4-inch piece of ginger, sliced 1.0 3.0
3.0-to-4-inch piece of ginger, sliced 3.0 3.0
1 5-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 1.0 5.0
5.0-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 5.0 5.0
1 5-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 1.0 5.0
5.0-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 5.0 5.0
1 5-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 1.0 5.0
5.0-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 5.0 5.0
1 5-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 1.0 5.0
5.0-to-7-pound leg of lamb, preferably at room temperature, shank removed if necessary 5.0 5.0
1 14-ounce box firm tofu, cut into dominoes 14.0 14.0
14.0-ounce box firm tofu, cut into dominoes 14.0 14.0
1 2-pound piece fresh pork belly, with s

In [14]:
def fix_abbreviations(s):
    """
    Converts instances of oz. and g. to ounce and gram respectively
    """
    columns = ["input", "unit"]
    for col in columns:
        # replace oz. with ounce
        if s[col] == s[col]:
            match = re.findall(r"([0-9])\s*oz\.*", s[col])
            if match:
                for m in match:
                    s[col] = re.sub(r"([0-9])\s*oz\.*", m + " ounce", s[col])
            # replace g. with gram
            match = re.findall(r"([0-9])\s*g([^a-z])", s[col])
            if match:
                for m in match:
                    s[col] = re.sub(
                        r"([0-9])\s*g([^a-z])", m[0] + " gram" + m[1], s[col]
                    )
            # replace tbsp with tablespoon
            match = re.findall(r"[Tt]bsp\.*", s[col])
            if match:
                for m in match:
                    s[col] = re.sub(r"[Tt]bsp\.*", "tablespoon", s[col])
            # replace tsp with teaspoon
            match = re.findall(r"[Tt]sp\.*", s[col])
            if match:
                for m in match:
                    s[col] = re.sub(r"[Tt]sp\.*", "teaspoon", s[col])
    return s

In [15]:
input_data = input_data.apply(fix_abbreviations, axis=1)

In [16]:
def fix_inconsistencies(row):
    """
    Fix various inconsistencies in the labels.
    """
    try:
        if ',' in row["name"]:
            name = row["name"].split(',',1)
            #print(name)
    except TypeError:
        print(row)  

In [17]:
input_data.head()

,input,name,qty,range_end,unit,comment
index,,,,,,
0,1.25 cups cooked and pureed fresh butternut sq...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,"1 medium-size onion, peeled and chopped",onion,1,0.0,NaN,"medium-size, peeled and chopped"
3,"2 stalks celery, chopped coarse",celery,2,0.0,stalk,chopped coarse
4,1.5 tablespoons vegetable oil,vegetable oil,1.5,0.0,tablespoon,NaN


In [18]:
input_data[(input_data['input'].str.contains('garlic')) & (input_data['name'] != "garlic")].apply(fix_inconsistencies, axis=1)


input        2 cloves garlic
name                     NaN
qty                        0
range_end                  0
unit                     NaN
comment                  NaN
Name: 120232, dtype: object
input        4 garlic cloves, minced
name                             NaN
qty                                0
range_end                          0
unit                             NaN
comment                          NaN
Name: 144161, dtype: object
input        Add cooled zucchini and pine nuts to food proc...
name                                                       NaN
qty                                                          0
range_end                                                    0
unit                                                       NaN
comment                                                    NaN
Name: 150489, dtype: object
input        1 small garlic clove puréed
name                                 NaN
qty                                    0
range_end          

index
185       None
238       None
773       None
782       None
1266      None
          ... 
178853    None
178993    None
178994    None
179009    None
179037    None
Length: 1380, dtype: object

In [19]:
training_data, test_data = train_test_split(input_data, test_size=0.2)

In [20]:
#training_data.to_pickle("../data/interim/crf_training_data.pickle")
#test_data.to_pickle("../data/interim/crf_test_data.pickle")

In [29]:
# Examining some remaining outliers in the data
input_data.qty.unique()

array([1.25, 1.0, 2.0, 1.5, 0.0, 6.0, 0.33, 0.5, 4.0, 3.0, 12.0, 13.0,
       0.25, 5.0, 0.75, '2.0', 0.13, 2.5, '6.0', 9.0, 18.0, 56.0, 8.0,
       '36.0', 3.25, 6.5, 10.0, 2.25, 7.0, 0.67, 4.5, 0.58, 32.0, '2.5',
       17.0, '3.0', 28.0, 30.0, 35.0, 1.67, 20.0, '4.0', 1.75, 16.0,
       '9.0', 9.5, 40.0, 21.0, 15.0, 24.0, 5.5, 69.0, 1.33, '750.0', 11.0,
       '8.5', '1.5', '28.0', '11.0', '3.5', '8.0', 60.0, '14.0', 25.0,
       3.5, 14.0, '10.0', '1.0', 0.38, '24.0', 7.33, 33.0, '68.0', '16.0',
       '13.0', 1.13, 68.0, '29.0', 26.0, '1500.0', 45.0, '19.0', 7.5,
       '20.0', 17.5, '4.5', '26.0', 22.0, '48.0', 11.67, '12.0', 13.5,
       6.33, 3.75, '15.0', 14.5, 2.67, '17.0', 2.75, 12.67, 36.0, 2.33,
       '14.5', '5.5', 0.83, 3.33, 27.0, 0.66, '7.0', '11.5', '5.0', 4.25,
       57.0, 34.0, 54.0, '6.5', '64.0', 15.5, 22.5, 63.0, '13.5', '18.0',
       '32.0', '10.75', 3.67, 80.0, 0.06, '288.0', '15.5', 19.5, '35.0',
       58.0, 49.0, '7.5', 48.0, 19.0, 12.33, 50.0, 1.66, '13.

In [22]:
input_data[(input_data['unit']=="ounces 1 stick")]

,input,name,qty,range_end,unit,comment
index,,,,,,
124063,4 ounces (1 stick) unsalted butter,unsalted butter,4,NaN,ounces 1 stick,NaN


In [26]:
def fix_inconsistencies(row):
    match = re.findall(r"(\w+)\s(sprigs?)", row["unit"])
    if match:
        for m in match:
            row["comment"] += " " + m[0]
            row["unit"] = m[1]
    return row


def fix_individual_rows(row):
    if row["input"] == "3 crushed red peppers":
        row["unit"] = float("nan")
        row["comment"] = "crushed"
    if row["input"] == "1 small-to-medium daikon radish (cut into 1-inch cubes)":
        row["comment"] = row["unit"] + " " + row["comment"] 
        row["unit"] = float("nan")
    if row["unit"] == "chopped":
        row["comment"] = "chopped"
        row["unit"] = float("nan")
    if row["input"] == "1 heaping teaspoon black peppercorns":
        row["comment"] = "heaping"
        row["unit"] = "teaspoon"
    if row["input"] == "a 10-pound piece of pork belly with the skin":
        row["qty"] = 10
        row["unit"] = "pound"
    if row["input"] == "1 long soft baguette or loaf Cuban bread":
        row["unit"] = float("nan")
        row["comment"] = 'long'
    if row["input"] == "2 long red chilies, seeded and finely sliced":
        row["unit"] = float("nan")
        row["comment"] = 'long ' + row["comment"]
    if row["input"] == "2 scant cups all-purpose flour":
        row["unit"] = "cups"
        row["comment"] = "scant"
    if row["input"] == "4 ounces (1 stick) unsalted butter":
        row["unit"] = "ounces"
        row["comment"] = "(1 stick)"
    return row
    
        
        

In [27]:
input_data = input_data.apply(fix_individual_rows, axis=1)

In [28]:
input_data = input_data.apply(fix_inconsistencies, axis=1)

TypeError: ('expected string or bytes-like object', 'occurred at index 2')

In [97]:
input_list = input_data["name"].apply(type)

In [98]:
input_list.unique()

array([<class 'str'>, <class 'float'>], dtype=object)